## Importamos el excel

Verificar Recursos del Sistema

In [1]:
import tensorflow as tf

print("GPUs disponibles:", len(tf.config.experimental.list_physical_devices('GPU')))


2024-07-11 19:44:53.154109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-07-11 19:44:53.154129: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


GPUs disponibles: 0


2024-07-11 19:44:54.752229: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-11 19:44:54.752558: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-07-11 19:44:54.752618: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-07-11 19:44:54.752672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared objec

In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. Número de GPUs:", torch.cuda.device_count())
    print("Nombre de la GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible.")


CUDA está disponible. Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 2070 SUPER


In [3]:
import os
import multiprocessing
import torch
import tensorflow as tf

def check_cpus():
    print("Número de CPUs (os.cpu_count()):", os.cpu_count())
    print("Número de CPUs (multiprocessing.cpu_count()):", multiprocessing.cpu_count())

def check_gpu_pytorch():
    if torch.cuda.is_available():
        print("CUDA está disponible. Número de GPUs:", torch.cuda.device_count())
        print("Nombre de la GPU:", torch.cuda.get_device_name(0))
    else:
        print("CUDA no está disponible en PyTorch.")

def check_gpu_tensorflow():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        print("GPUs disponibles en TensorFlow:", len(gpus))
        for gpu in gpus:
            print("Nombre de la GPU:", gpu.name)
    else:
        print("CUDA no está disponible en TensorFlow.")

if __name__ == "__main__":
    check_cpus()
    check_gpu_pytorch()
    check_gpu_tensorflow()


Número de CPUs (os.cpu_count()): 12
Número de CPUs (multiprocessing.cpu_count()): 12
CUDA está disponible. Número de GPUs: 1
Nombre de la GPU: NVIDIA GeForce RTX 2070 SUPER
CUDA no está disponible en TensorFlow.


Cargar y Preprocesar Datos

In [4]:
import pandas as pd

# Ruta al archivo Excel
file_path = '/home/jovyan/work/transcripciones_frases_v5.xlsx'

# Cargamos el archivo Excel
df = pd.read_excel(file_path)

# Verificar que las columnas existen
assert 'Texto Original' in df.columns, "La columna 'Texto Original' no existe en el archivo Excel"
assert 'Texto Corregido' in df.columns, "La columna 'Texto Corregido' no existe en el archivo Excel"

df.head()


,Texto Original,Texto Corregido
0,esto es importante para cada adulto o cada per...,esto es importante para cada adulto o cada per...
1,no salió todo como queríamos fue un buen año m...,"no salió todo como queríamos, fue un buen año...."
2,a lo largo de este top verás retratadas siete ...,a lo largo de este top verás retratadas siete ...
3,te prevengo querida audiencia los próximos min...,"te prevengo, querida audiencia, los próximos m..."
4,número siete Benjamin Solari parravicini fue u...,número siete: Benjamín Solari Parravicini fue ...


Dividir Datos en Entrenamiento y Validación

In [5]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.head()

,Texto Original,Texto Corregido
1769,Y hoy en día y más aún con el futuro que se vi...,"y hoy en día, y más aún con el futuro que se v..."
1915,Muchos de estos omicidios se dan cuando ven qu...,muchos de estos homicidios se dan cuando ven q...
631,este Wow ha sido de mis favoritos sobre todo p...,"este, wow, ha sido de mis favoritos, sobre tod..."
1814,Bien pues una de estas ramas importantes en el...,"bien, pues una de estas ramas importantes en e..."
1670,"¡Wow, ordenador, qué buscaba! Estaba intentand...","¡Wow, ordenador, qué buscaba! Estaba intentand..."


In [6]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargar el Modelo y el Tokenizador

Usaremos el modelos T5. Tokenizamos antes de introducir el texto en el modelo con T5Tokenizer 
Se puede usar 't5-base' o 't5-large' según lo preciso (más tiempo de entrenamiento) que queramos entrenar el modelo 

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq


# Cargar el tokenizador y el modelo
# model_name = 't5-base'  # Puedes usar 't5-small', 't5-large', etc.
model_name = ("vgaraujov/t5-base-spanish")
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
# Definir prefijo y longitudes máximas
prefix = "corrija: "
max_input_length = 150
max_target_length = 150

# Función de preprocesamiento
def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples['Texto Original']]
    targets = [ex for ex in examples['Texto Corregido']]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Aplicar preprocesamiento a los datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)


Map:   0%|          | 0/2801 [00:00<?, ? examples/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 701/701 [00:00<00:00, 3926.86 examples/s]


In [9]:
# Imprimir la estructura del dataset después del preprocesamiento
print("Estructura del dataset de entrenamiento después del preprocesamiento:")
print(train_dataset)
print("Ejemplo de datos preprocesados:")
print(train_dataset[0])

print("Estructura del dataset de validación después del preprocesamiento:")
print(val_dataset)
print("Ejemplo de datos preprocesados:")
print(val_dataset[0])

Estructura del dataset de entrenamiento después del preprocesamiento:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2801
})
Ejemplo de datos preprocesados:
{'input_ids': [15586, 584, 33, 94, 253, 10, 121, 9, 27, 350, 16, 11, 550, 7, 15, 750, 19, 55, 322, 179, 3, 749, 1106, 9, 147, 8, 12, 60, 3, 13, 135, 8, 57, 25, 31569, 22, 23, 1944, 35, 10, 51, 9081, 31, 16352, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [10]:
import evaluate
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from datasets import Dataset
import numpy as np
import torch


# Definir la métrica
metric = evaluate.load('sacrebleu')

# Definir la función postprocess_text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

# Definir la función compute_metrics
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    torch.cuda.empty_cache()
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    torch.cuda.empty_cache()
    
    return result

# Definir una clase personalizada para el Trainer
class CustomSeq2SeqTrainer(Seq2SeqTrainer):
    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval", max_new_tokens=128):
        self.model.config.max_length = max_new_tokens
        return super().evaluate(eval_dataset=eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)

# Añadir logs para verificar los datos antes del entrenamiento
def check_data(dataset):
    for example in dataset:
        input_ids = example.get("input_ids", None)
        labels = example.get("labels", None)
        if input_ids is None or labels is None:
            print("Error: 'input_ids' o 'labels' no encontrados en el ejemplo", example)
            continue
        if any(id >= tokenizer.vocab_size or id < 0 for id in input_ids):
            print("Error: input_ids fuera de rango", input_ids)
        if any(id >= tokenizer.vocab_size or (id < 0 and id != -100) for id in labels):
            print("Error: labels fuera de rango", labels)

# Verificar datos de entrenamiento y evaluación
check_data(train_dataset)
check_data(val_dataset)


Configurar y Entrenar el Modelo

In [11]:
# Configurar y Entrenar el Modelo
training_args = Seq2SeqTrainingArguments(
    output_dir='/home/jovyan/results',  # Directorio de salida
    num_train_epochs=8,                # Épocas
    per_device_train_batch_size=2,      # Tamaño del lote de entrenamiento
    per_device_eval_batch_size=2,       # Tamaño del lote de evaluación
    warmup_steps=100,                    # Pasos de calentamiento
    weight_decay=0.00001,                 # Desintegración del peso
    learning_rate=2e-5,
    logging_dir='/home/jovyan/logs',    # Directorio de logs
    eval_strategy="epoch",              # Estrategia de evaluación
    save_strategy="epoch",              # Estrategia de guardado
    fp16=True,                          # Entrenamiento en punto flotante de 16 bits
    save_total_limit=1,
    predict_with_generate=True,
    generation_max_length=300,           # Longitud máxima de generación (puedes ajustarlo)
    generation_num_beams=10              # Número de haces de búsqueda
)

# Definir el entrenador personalizado
trainer = CustomSeq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Entrenar el modelo
trainer.train()

Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1364: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


IndexError: piece id is out of range.

In [ ]:
model_save_path = '/home/jovyan/work/correct_transcription_model_base_v5'
tokenizer_save_path = '/home/jovyan/work/correct_transcription_tokenizer_base_v5'

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

In [ ]:
import numpy as np
import evaluate
from torch.utils.data import DataLoader

# Cargar la métrica
metric = evaluate.load("sacrebleu")

def compute_metrics(preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return result

def evaluate_model(model, dataloader):
    model.eval()
    preds, labels = [], []
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)
        labels_batch = batch['labels'].to(model.device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_target_length)
        
        preds.extend(outputs.cpu().numpy())
        labels.extend(labels_batch.cpu().numpy())
    
    metrics = compute_metrics(preds, labels)
    return metrics

val_dataloader = DataLoader(val_dataset, batch_size=2, collate_fn=data_collator)

# Mover el modelo a la GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Evaluar el modelo
metrics = evaluate_model(model, val_dataloader)
print(metrics)

In [ ]:
def generate_predictions(model, dataloader, tokenizer, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()
    predictions = []
    references = []
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_target_length, num_beams=4, early_stopping=True)
        
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        predictions.extend(preds)
        references.extend(refs)
    
    return predictions, references

# Crear DataLoader
val_dataloader = DataLoader(val_dataset, batch_size=2, collate_fn=data_collator)

# Evaluar el modelo
val_predictions, val_references = generate_predictions(model, val_dataloader, tokenizer)

# SacreBLEU y ROUGE esperan listas de listas para referencias
references = [[ref] for ref in val_references]

metric = evaluate.load("sacrebleu")
bleu_score = metric.compute(predictions=val_predictions, references=references)

print("BLEU score:", bleu_score)

Generar Predicciones y Evaluar el Modelo

In [ ]:
# Generar predicciones y evaluar el modelo
def generate_predictions(model, dataloader, tokenizer, max_length=96, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()
    predictions = []
    references = []
    
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=4, early_stopping=True)
        
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        refs = tokenizer.batch_decode(labels, skip_special_tokens=True)
        
        predictions.extend(preds)
        references.extend(refs)
    
    return predictions, references

val_dataloader = DataLoader(val_dataset, batch_size=2, collate_fn=data_collator)
val_predictions, val_references = generate_predictions(model, val_dataloader, tokenizer)
references = [[ref] for ref in val_references]

metric = evaluate.load("sacrebleu")
bleu_score = metric.compute(predictions=val_predictions, references=references)
print("BLEU score:", bleu_score)

Generar Predicciones y Evaluar el Modelo

In [ ]:
def generate_predictions(model, tokenizer, dataset, max_length=96, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()
    predictions = []
    for i in range(len(dataset)):
        inputs = tokenizer(dataset[i]['Texto Original'], return_tensors="pt", padding=True, truncation=True, max_length=max_length)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        outputs = model.generate(inputs['input_ids'], max_length=max_length, num_beams=4, early_stopping=True)
        pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(pred_text)
    return predictions

val_predictions = generate_predictions(model, tokenizer, val_dataset)
references = val_df['Texto Corregido'].tolist()

# SacreBLEU y ROUGE esperan listas de listas para referencias
references = [[ref] for ref in references]

rouge_score = metric.compute(predictions=val_predictions, references=references)
bleu_score = metric.compute(predictions=[pred.split() for pred in val_predictions], references=[[ref.split()] for ref in references])

print("ROUGE score:", rouge_score)
print("BLEU score:", bleu_score)